# Tarea 4 - Optimización
### Gradiente Conjugado Lineal y Gradiente Conjugado No Lineal.
#### Por: Gustavo Hernández Angeles


## **Preparación**

In [19]:
import numpy as np
from time import perf_counter
np.random.seed(42)
np.set_printoptions(precision=4)

## **Gradiente Conjugado Lineal**

### **Funciones de utilidad**

Generamos la matriz $\vec{A}$ de tamaño NxN simétricas y definidas positivas.  
Generamos el vector $\vec{b}$ de tamaño Nx1.

In [20]:
def generar_MatrizVector(n):
    todos_positivos = False
    while not todos_positivos:
        A = np.random.rand(n,n)
        A = 0.5*(A + A.T) # Simétrica
        eigenvalores = np.linalg.eigvals(A)
        todos_positivos = eigenvalores.min() > 0 # Positiva definida
    return A, np.random.rand(n,1)

#### Gradiente Conjugado Lineal

In [21]:
def gc_lineal(A, b, x_0, epsilon, max_iter):
    x_k = x_0.copy()
    r_k = A.dot(x_0) - b
    p_k = -r_k

    for i in range(max_iter):
        alfa_k = - (r_k.T.dot(p_k)) / (p_k.T.dot(A).dot(p_k))
        x_k = x_k + alfa_k*p_k
        r_k = A.dot(x_k) - b
        beta_k = r_k.T.dot(A).dot(p_k) / p_k.T.dot(A).dot(p_k)
        p_k = -r_k + beta_k*p_k
        
        # Evalua la convergencia
        convergencia = max(abs(r_k)) < epsilon
        if convergencia:
            print(f"El método converge en la iteracion: {i+1}.")
            break
    
    if not convergencia:
        print(f"El método no converge en {i+1} iteraciones.")
    
    return x_k

### **Benchmark**

#### Pruebas

In [22]:
n = 5
A, b = generar_MatrizVector(n)
x_0 = np.random.rand(n,1)
epsilon = 1e-4
max_iter= n

In [23]:
x_sol = gc_lineal(A, b, x_0, epsilon,max_iter)

print(f"Solucion real:\n",np.linalg.solve(A,b).reshape(1,-1))
print(f"Solución aprox:\n",x_sol.reshape(1,-1))

El método converge en la iteracion: 5.
Solucion real:
 [[ 0.9137  0.7087  0.9457 -1.9344  0.1834]]
Solución aprox:
 [[ 0.9137  0.7087  0.9457 -1.9344  0.1834]]


## **Gradiente Conjugado No Lineal (Fletcher-Reeves)**

### **Funciones de utilidad**

#### Definimos funciones de prueba.
* Esfera
* Rosenbrock
* Beale

In [24]:
def Esfera(x):
    """
    f(x) = \\sum x_i^2
    """
    
    return np.sum(x**2)


def Rosenbrock(x):
    terminos = [
        100 * (x[i + 1] - x[i] ** 2) ** 2 + (1 - x[i]) ** 2
        for i in range(len(x) - 1)
    ]
    terminos = np.array(terminos)
    return np.sum(terminos)


def Beale(x):
    assert len(x) == 2, f"La función Beale es de dos variables, se introdujeron {len(x)}."
    (x1, x2) = x
    return (
        (1.5 - x1 + x1 * x2) ** 2
        + (2.25 - x1 + x1 * x2**2) ** 2
        + (2.625 - x1 + x1 * x2**3) ** 2
    )

#### Recuperamos el algoritmo de Backtracking de la tarea 2.

In [25]:
def backtracking(f, gradiente, x, p, alfa, h, rho = 0.5, c_1 = 10e-4, max_iter = 50):
    for i in range(max_iter):
        if f(x+p*alfa) <= f(x) + c_1 * alfa * np.dot(p, gradiente):
            return alfa
        alfa = alfa*rho
                
    print(f"No se cumplió la condición con {max_iter} iteraciones.")
    return alfa

#### Calculo de gradiente

In [26]:
def grad_f(x, f, h):
    """
    Input:
        f: Función.
        x: Punto a evaluar (numpy array).
        h: Espaciamiento para el cálculo del gradiente.
        
    Output:
        grad: Valor del gradiente (numpy array).
    """
    
    # Inicializa el gradiente
    grad = np.zeros(len(x))

    # Itera sobre cada componente
    for i in range(len(x)):
        # Copia de x
        x_i = np.copy(x)

        # Se suma el espaciamiento solo en la i-esima componente
        x_i[i] = x_i[i] + h
        
        # Se calcula la i-esima componente del gradiente
        grad[i] = (f(x_i) - f(x)) / h
    return grad

#### GC No Lineal (Fletcher-Reeves)

In [27]:
def gc_nolineal(f, x, max_iter, epsilon, h):
    x_k = x.copy()
    grad = grad_f(x, f, h)
    p_k = -grad
    alfa_k = 1
    
    for i in range(max_iter):
        alfa_k = backtracking(f,grad,x_k,p_k,alfa_k,h)
        x_k += alfa_k * p_k
        grad_1 = grad_f(x_k, f, h)
        beta_k = (grad_1.T.dot(grad_1)) / (p_k.T.dot(p_k))
        p_k = -grad_1 + beta_k*p_k
    
        # Evalua la convergencia
        convergencia = max(abs(p_k)) < epsilon
        if convergencia:
            print(f"La función {f.__name__} converge en la iteracion: {i}")
            break
    
    if not convergencia:
        print(f"No se cumplio la convergencia en {max_iter} iteraciones.")

    return x_k

### **Benchmark**

#### Función Esfera

In [28]:
n = 4
x = np.array(n*[4], dtype=float)
alfa = 1
max_iter = 15_000
epsilon = 10e-6
h = 10e-6

inicio = perf_counter()
xsol = gc_nolineal(Esfera, x, max_iter, epsilon, h)
final = perf_counter()

# Comparación con resultado real
solucion = np.array(n*[0], dtype=float)

print(f"Tiempo de ejecución: {(final-inicio) * 1e3:.5f} ms")
print(f"Magnitud del vector error: {np.linalg.norm(xsol - solucion)}")
print(f"Solución encontrada: {xsol}")



La función Esfera converge en la iteracion: 0
Tiempo de ejecución: 0.65290 ms
Magnitud del vector error: 9.99987560135196e-06
Solución encontrada: [-4.9999e-06 -4.9999e-06 -4.9999e-06 -4.9999e-06]


#### Función de Rosenbrock

In [29]:
n = 4
x = np.array(4*[2.5], dtype=float)
alfa = 1
max_iter = 15_000
epsilon = 10e-6
h = 10e-6

inicio = perf_counter()
xsol = gc_nolineal(Rosenbrock, x, max_iter, epsilon, h)
final = perf_counter()


# Comparación con resultado real
solucion = np.array(n*[1], dtype=float)

print(f"Tiempo de ejecución: {(final-inicio) * 1e3:.5f} ms")
print(f"Magnitud del vector error: {np.linalg.norm(xsol - solucion)}")
print(f"Solución encontrada: {xsol}")

La función Rosenbrock converge en la iteracion: 13964
Tiempo de ejecución: 1843.89360 ms
Magnitud del vector error: 0.009193600574092933
Solución encontrada: [0.999 0.998 0.996 0.992]


#### Función de Beale


In [30]:
x = np.array([2,2], dtype=float)
alfa = 1
max_iter = 15_000
epsilon = 10e-6
h = 10e-6

inicio = perf_counter()
xsol = gc_nolineal(Beale, x, max_iter, epsilon, h)
final = perf_counter()



# Comparación con resultado real
solucion = np.array([3, 0.5], dtype=float)

print(f"Tiempo de ejecución: {(final-inicio) * 1e3:.5f} ms")
print(f"Magnitud del vector error: {np.linalg.norm(xsol - solucion)}")
print(f"Solución encontrada: {xsol}")

La función Beale converge en la iteracion: 5170
Tiempo de ejecución: 229.71260 ms
Magnitud del vector error: 0.000257087253322964
Solución encontrada: [2.9998 0.4999]
